In [19]:
import simplejson, urllib
import geopandas
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import googlemaps

from time import sleep as sleep
import datetime as dt
from datetime import timedelta as td
from random import randrange
import json
import signal
        

In [18]:


class DriveTimeSearcher:
    # constructor for this class
    def __init__(self, api_key_filename, apartment_list_filename, destination_list_filename, output_file):
        # load api key from file
        f_api=open(api_key_filename,)
        api_key = f_api.readline()
        f_api.close()

        # start googlemaps client
        self.gmaps = googlemaps.Client(key=api_key)

        self.apartment_list_filename = apartment_list_filename
        self.destination_list_filename = destination_list_filename
        self.output_file = output_file

        # load apartment list from file
        self.apartment_list = self.load_aparments()

        # parse destination list from file
        self.destination_list = self.load_destinations() 

        # initialize the results dictionary
        self.on_reset()
        
        self.start_time = dt.datetime.now()
        self.spin(td(days=7))

    # will come back to this later
    def spin(self,spintime):
        # signal handler to call save_results if interrupted
        signal.signal(signal.SIGINT, self.save_results)
        
        # get current day of the week
        current_day_of_week = dt.datetime.now().weekday()

        # spin until spintime is reached
        while dt.datetime.now() < self.start_time + spintime:
            # check if the time is right to query the next trip
            for start_address in self.apartment_list:
                for destination in [dest for dest in self.destination_list if current_day_of_week in dest['weekdays']]:
                    # get the next outbound query time
                    outbound_query_times = self.results[start_address][destination]['outbound_query_times']
                    if len(outbound_query_times) > 0:
                        next_outbound_query_time = outbound_query_times[0]
                        if dt.datetime.now() > next_outbound_query_time:
                            # query the next trip
                            self.query_trip(start_address, destination, outbound=True)
                            # remove the query time from the list
                            self.results[start_address][destination]['outbound_query_times'].pop(0)
                    
                    # get the next homebound query time
                    homebound_query_times = self.results[start_address][destination]['homebound_query_times']
                    if len(homebound_query_times) > 0:
                        next_homebound_query_time = homebound_query_times[0]
                        if dt.datetime.now() > next_homebound_query_time:
                            # query the next trip
                            self.query_trip(start_address, destination, outbound=False)
                            # remove the query time from the list
                            self.results[start_address][destination]['homebound_query_times'].pop(0)

            # sleep for 1 second
            sleep(1)


    # a function to save the results to a file
    def save_results(self):
               
        # save raw results to file
        with open(self.output_file, 'w') as outfile:
            json.dump(self.results, outfile,default=str)


    # a function to initialize theoretical departure times, and generate random query times.
    # will be called at script startup, at the beginning of each day, and if the config files change  
    def on_reset(self):
    
        self.save_results()

        # initialize the results dictionary
        self.results = {}
        for start_address in self.apartment_list:
            self.results[start_address] = {}
            for destination in self.destination_list:
                self.results[start_address][destination] = {}

        # get the theoretical departure times from each apartment to each destination
        for start_address in self.apartment_list:
            for destination in self.destination_list:
                # parse arrival times from the destination list
                arrival_times = self.destination_list[destination]['arrival_times']
                if arrival_times != None:
                    for arrival_time in arrival_times:
                        # get the theoretical departure time
                        theoretical_departure_time = self.get_theoretical_departure_time(start_address, destination, arrival_time)
                        self.results[start_address][destination][arrival_time] = theoretical_departure_time

                # get the theoretical departure time
                theoretical_departure_time = self.get_theoretical_departure_time(start_address, destination, dt.datetime.now())
                self.results[start_address][destination]['theoretical_departure_time'] = theoretical_departure_time

        # generate randomized departure times for each trip, for both apartment to dest and dest to apartment
        for start_address in self.apartment_list:
            for destination in self.destination_list:
                # get the theoretical departure time
                self.results[start_address][destination]['outbound_query_times']=[]
                
                theoretical_departure_time = self.results[start_address][destination]['theoretical_departure_time']
                if theoretical_departure_time != None:
                    samples_per_day = destination['samples_per_day']
                    for i in range(samples_per_day):
                        # generate a random departure time within 
                        random_departure_time = theoretical_departure_time + td(seconds=randrange(-30*60,60*30))
                        self.results[start_address][destination]['outbound_query_times'].append(random_departure_time)
                    # also query the theoretical departure time
                    self.results[start_address][destination]['outbound_query_times'].append(theoretical_departure_time)

                    # sort the list of outbound query times
                    self.results[start_address][destination]['outbound_query_times'].sort()
                else:
                    # use the given time range
                    outbound_departure_time_range = destination['outbound_departure_time_range']
                    samples_per_day = destination['samples_per_day']
                    for i in range(samples_per_day):
                        # generate a random departure time within range
                        random_departure_time = outbound_departure_time_range[0] + td(seconds=randrange(0,(outbound_departure_time_range[1]-outbound_departure_time_range[0]).seconds))
                        self.results[start_address][destination]['outbound_query_times'].append(random_departure_time)

                #do the same for the homebound trips
                self.results[start_address][destination]['homebound_query_times']=[]
                homebound_departure_time_range = destination['homebound_departure_time_range']
                samples_per_day = destination['samples_per_day']
                for i in range(samples_per_day):
                    # generate a random departure time within range
                    random_departure_time = homebound_departure_time_range[0] + td(seconds=randrange(0,(homebound_departure_time_range[1]-homebound_departure_time_range[0]).seconds))
                    self.results[start_address][destination]['homebound_query_times'].append(random_departure_time)

                # sort the list of outbound query times
                self.results[start_address][destination]['homebound_query_times'].sort()


    # load apartment list from file
    # returns a list of apartment addresses (strings)
    def load_aparments(self, verbose =False):
        f_ap=open(self.apartment_list_filename,)
        start_addresses = json.load(f_ap)
        f_ap.close()
        if verbose:
            print("loaded start addresses:")
            for s in start_addresses:
                print('\t',s) 

        return start_addresses
    
    # load a json file containing description of each aparment
    # returns a list of dictionaries
    def load_destinations(self, verbose =False):
        f_dest=open(self.destination_list_filename,)
        destinations = json.load(f_dest)
        f_dest.close()

        # for every destination, if it has an arrival time, convert it to a datetime object
        for destination in destinations:
            if destinations[destination]['arrival_times'] != None:
                for i in range(len(destinations[destination]['arrival_times'])):
                    destinations[destination]['arrival_times'][i] = dt.datetime.strptime(destinations[destination]['arrival_times'][i],'%H:%M:%S').time()

        # for every destination, if it has an outbound departure time range, convert it to a datetime object
        for destination in destinations:
            if destinations[destination]['outbound_departure_time_range'] != None:
                for i in range(len(destinations[destination]['outbound_departure_time_range'])):
                    destinations[destination]['outbound_departure_time_range'][i] = dt.datetime.strptime(destinations[destination]['outbound_departure_time_range'][i],'%H:%M:%S').time()
        
        # for every destination, if it has an homebound departure time range, convert it to a datetime object
        for destination in destinations:
            if destinations[destination]['homebound_departure_time_range'] != None:
                for i in range(len(destinations[destination]['homebound_departure_time_range'])):
                    destinations[destination]['homebound_departure_time_range'][i] = dt.datetime.strptime(destinations[destination]['homebound_departure_time_range'][i],'%H:%M:%S').time()

        if verbose:
            print("loaded destinations:")
            for d in destinations:
                # print name of destination
                print('\t',d['name'])

                # print every other attribute in the dictionary
                for key in d:
                    if key != 'name':
                        print('\t\t',key,':',d[key])  

        return destinations


    # a function to query the google maps api for the expected time to leave for a destination
    def get_theoretical_departure_time(self, start_address, destination, arrival_time):
        # get the duration of the trip
        directions_result = self.gmaps.directions(
            start_address, 
            destination, 
            mode="driving", 
            arrival_time=arrival_time)

        # get the duration in traffic
        duration_in_traffic = directions_result[0]['legs'][0]['duration_in_traffic']['value']

        # calculate departure respecting traffic
        departure_time_in_traffic = arrival_time - td(seconds=duration_in_traffic)

        # return the trip info
        return {
            'duration_in_traffic':duration_in_traffic,
            'departure_time_in_traffic':departure_time_in_traffic
        }

    # get the duration of a trip from a start address to a destination
    # returns a dictionary containing the trip duration, mileage, and time of arrival, average velocity
    def live_query_trip(self, start_address, destination):
        # get the duration of the trip
        departure_time = dt.datetime.now() 
        directions_result = self.gmaps.directions(
            start_address, 
            destination, 
            mode="driving", 
            departure_time=departure_time)

        # save the directions result to a map which we can use later
        if self.results[start_address] == None:
            self.results[start_address] = {}
        if self.results[start_address][destination] == None:
            self.results[start_address][destination] = {}
        
        self.results[start_address][destination][departure_time] = directions_result

        # get the duration of the trip
        duration = directions_result[0]['legs'][0]['duration']['value']
        duration = td(seconds=duration)

        # get the duration in traffic
        duration_in_traffic = directions_result[0]['legs'][0]['duration_in_traffic']['value']

        # get the distance of the trip
        distance = directions_result[0]['legs'][0]['distance']['value']
        distance = distance/1609.34  # ! convert to miles apparently, not sure about this copilot...

        # calculate arrival time as duration + departure_time
        arrival_time = departure_time + duration

        # calculate arrival respecting traffic
        arrival_time_in_traffic = departure_time + td(seconds=duration_in_traffic)

        # calculate average mph from duration and distance
        avg_velocity = distance/duration.total_seconds()*3600
        
        # return the trip info
        ret = {
            'duration':duration,
            'duration_in_traffic':duration_in_traffic,
            'distance':distance, 
            'arrival_time':arrival_time, 
            'avg_velocity':avg_velocity, 
            'avg_velocity_in_traffic':distance/duration_in_traffic*3600,
            'arrival_time_in_traffic':arrival_time_in_traffic
        }
        print(ret)
    
    def test(self):
        # test the get_trip_duration function
        test_start_address = '1600 Amphitheatre Parkway, Mountain View, CA'
        test_destination = 'San Francisco, CA'
        test_result = self.query_trip(test_start_address, test_destination)
        return test_result



In [57]:
def load_aparments(a_fname="apartments.json", verbose =False):
    f_ap=open(a_fname,)
    start_addresses = json.load(f_ap)
    f_ap.close()
    if verbose:
        print("loaded start addresses:")
        for s in start_addresses:
            print('\t',s)

loaded start addresses:
	 2901 Barton Skyway, Austin, TX 78746


In [69]:
from random import randrange
print([randrange(-5,5) for i in range(5)])

[-2, 3, -5, 0, -1]


In [ ]:
a_fname="apartments.json"
dest_f_name='destinations.json'
key_file = 'key.txt'


13:11:08.636802


In [15]:
def add_delta_to_time(t,delta):
    a= (dt.datetime.combine(dt.date(1,1,1),t) + delta)
    a.time()
    return a.time()

print(add_delta_to_time(dt.datetime.now().time(),td(hours = 0)))


13:11:08.636802
